In [1]:

import pyupbit
print("\n\nAutobit.\n\n")

try:
    AK = "XNZxdYXZD5pjqab5fzJLnx9iVWZxuJYPj59ZWxZE"
    SK = "P8QrzR4byKVxWeWa0NTJJNdxsB4UPi2Y6TqKveZ5"
    Upbit_Object = pyupbit.Upbit(AK, SK)
except:
    print("Upbit_Object Create Fail. Please Check API KEY . EXIT") 
    exit() 


print(
    "현재 잔액 = " , 
    Upbit_Object.get_balances()[0]['balance'], 
    Upbit_Object.get_balances()[0]['currency'], 
)

if (Upbit_Object.get_balances()[0]['balance'] == 'KRW' and 
    Upbit_Object.get_balances()[0]['currency'] < 5000): 
    print("KRW Balance < 5000. Please Charge KRW. EXIT") 
    exit() 




Autobit.


현재 잔액 =  54757.94964477 KRW


In [2]:

# Current Price
def Current_Price(Ticker):
    Price = -1
    try:
        Price = pyupbit.get_current_price(Ticker)
        while True :
            if Price != -1 : break 
        if Price == 0:
            print("FNC(Current_Price)_ Price==0 . EXIT") 
            exit() 
    except:
        print("FNC(Current_Price)_ Except . EXIT") 
        exit() 
    return Price

In [3]:

# Current Balance
def Current_Balance(Ticker):
    Balance = -1
    try:
        while True :
            Balance = Upbit_Object.get_balance(Ticker)
            if Balance != -1 : break 
    except:
        print("FNC(Current_Balance)_ Except . EXIT") 
        exit() 
    return Balance

In [4]:

# OHLCV 시가(Open), 고가(High), 저가(Low), 종가(Close), 거래량(Volume)
def Get_OHLCV(Ticker, Interval, Count):
    try:
        Price = pyupbit.get_ohlcv(Ticker, interval=Interval, count=Count)
        while True :
            if len(Price) == Count : break 
    except:
        print("FNC(Current_OHLCV)_ Except . EXIT") 
        exit()
    return Price

In [5]:

# Get_Tickers (KRW, BTC, USDT)
def Get_Tickers(Unit):
    try:
        Tickers = pyupbit.get_tickers(fiat=Unit)
        if len(Tickers) == 0:
            print("FNC(Get_Tickers)_ Lenth==0 . EXIT") 
            exit() 
    except:
        print("FNC(Get_Tickers)_ Except . EXIT") 
        exit() 
    return Tickers


In [6]:

import ta
def BBND(DF, W):
    DF['Upper'] = ta.volatility.bollinger_hband(close=DF['close'], window=W, window_dev=2)
    DF['Lower'] = ta.volatility.bollinger_lband(close=DF['close'], window=W, window_dev=2)
    DF  = DF[W:] 
    return DF


In [7]:

import ta
def MACD(DF, W_F, W_S, W_Si):
    DF['MACD'] = ta.trend.macd_signal(close=DF['close'], window_slow = W_S, window_fast = W_F, window_sign = W_Si)
    DF['MACD_S'] = ta.trend.macd(close=DF['close'], window_slow = W_S, window_fast = W_F)
    DF  = DF[W_S+3:] 
    return DF

In [8]:

import ta
def RSI(DF, W):
    DF['RSI'] = ta.momentum.rsi(close=DF['close'], window = W)
    DF  = DF[W:] 
    return DF

In [9]:

import ta
def MA(DF, W):
    DF['MA'] = ta.trend.sma_indicator(close=DF['close'], window = W)
    DF  = DF[W:] 
    return DF

In [10]:

def Get_DF(T, OHLCV_I, OHLCV_P, MACD_F, MACD_S, MACD_Si, BBND_W, RSI_W, MA_W):
    DF = Get_OHLCV(T, OHLCV_I, OHLCV_P + MACD_S + BBND_W + MA_W)
    DF = BBND(DF, BBND_W)
    DF = MACD(DF, MACD_F, MACD_S, MACD_Si)
    DF = RSI(DF, RSI_W)
    DF = MA(DF, MA_W)
    return DF

In [11]:
import numpy as np 
import pandas as pd 
import time
from fbprophet import Prophet
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

Importing plotly failed. Interactive plots will not work.


In [ ]:
Gain_ = Rate_ = Gain = Rate = 0

Buy_Price = 5500 # S

Deal_Commission = 0.05 # %

for Ticker in Get_Tickers("KRW"): 
    globals()['BT_{}'. format(Ticker)], globals()['BP_{}'. format(Ticker)] = "0", 0
    globals()['ST_{}'. format(Ticker)], globals()['SP_{}'. format(Ticker)] = "0", 0
    globals()['Gain_{}'. format(Ticker)], globals()['Rate_{}'. format(Ticker)] = 0, 0

Tickers = []
for Ticker in Get_Tickers("KRW"):
    if (Current_Price(Ticker) > 0) : Tickers.append(Ticker)
    time.sleep(0.1)
Tickers.remove('KRW-NU')
Buy_Ticker = 0

while True:

    for Ticker in Tickers: #Get_Tickers("KRW") ["KRW-BTC"]

        DF = Get_DF(Ticker, "minute10", 20     , 10    , 20    , 5      , 20    , 5    , 20  )
            #Get_DF(T     , OHLCV_I  , OHLCV_P, MACD_F, MACD_S, MACD_Si, BBND_W, RSI_W, MA_W)
        BT, BP = globals()['BT_{}'. format(Ticker)], globals()['BP_{}'. format(Ticker)]
        ST, SP = globals()['ST_{}'. format(Ticker)], globals()['SP_{}'. format(Ticker)]

        # BUY RULE # 1
        # MAX LOW   
        if (
            (DF['close'] < DF['Lower']).iloc[-1] and
            (DF['Lower'].iloc[-1] > Current_Price(Ticker)) and
            (DF['close'].iloc[-1] > Current_Price(Ticker)) and 
            (DF['RSI'].iloc[-1] < 20) and 
            #(DF['RSI'].iloc[-2] < DF['RSI'].iloc[-1]) and 
            (DF['MA'].iloc[-1] > Current_Price(Ticker)) and 
            #(DF['MACD'].iloc[-1] < 0) and 
            #(DF['MACD_S'].iloc[-1] < 0) and 
            ((int)(float(Upbit_Object.get_balances()[0]['balance'])) > Buy_Price + (Buy_Price * 0.01 * Deal_Commission) ) and
            (Current_Balance(Ticker) == 0)
            ):
                BT = DF.index[-1]
                BP = Buy_Price + (Buy_Price * 0.01 * Deal_Commission)
                print("매수", BT, Ticker, Rate_, BP)

                #BUY
                Upbit_Object.buy_market_order(Ticker, Buy_Price)

        # SELL RULE # 1
        # BBAND_HIGHSTRIKE   
        if (
            (DF['close'] > DF['Upper']).iloc[-1] and 
            (Current_Balance(Ticker) > 0)
            #(BP < Current_Balance(Ticker))
            ):

                ST = DF.index[-1]
                SP = Current_Balance(Ticker)
                
                Gain_ = SP - BP
                Rate_ = (((Current_Balance(Ticker) + (Current_Balance(Ticker) * 0.01 * Deal_Commission)) / BP) * 100 - 100)

                Gain += Gain_
                Rate += Rate_

                print("매도", ST, Ticker, Current_Balance(Ticker), "%\n Total Rate", Rate, "%", "%\n Rule 1")

                #SELL
                Upbit_Object.sell_market_order(Ticker, Current_Balance(Ticker))

        # SELL RULE # 2
        # MACD_UPTO0
        elif ( 
             (DF['MACD'].iloc[-1] > 0) and 
             (DF['MACD_S'].iloc[-1] > 0) and   
             (Current_Balance(Ticker) > 0)
             #(BP < Current_Balance(Ticker))
            ):

                ST = DF.index[-1]
                SP = Current_Balance(Ticker)

                Gain_ = SP - BP
                Rate_ = (((Current_Balance(Ticker) + (Current_Balance(Ticker) * 0.01 * Deal_Commission)) / BP) * 100 - 100)

                Gain += Gain_
                Rate += Rate_

                print("매도", ST, Ticker, Current_Balance(Ticker), "%\n Total Rate", Rate, "%", "%\n Rule 2")

                #SELL
                Upbit_Object.sell_market_order(Ticker, Current_Balance(Ticker))

        # SELL RULE # 3
        # MACD_DIFF
        elif ( 
            (DF['MACD'].iloc[-1] < DF['MACD_S'].iloc[-1]) and
            (Current_Balance(Ticker) > 0)
            #(BP < Current_Balance(Ticker))
            ):

                ST = DF.index[-1]
                SP = Current_Balance(Ticker)

                Gain_ = SP - BP
                Rate_ = (((Current_Balance(Ticker) + (Current_Balance(Ticker) * 0.01 * Deal_Commission)) / BP) * 100 - 100)

                Gain += Gain_
                Rate += Rate_

                print("매도", ST, Ticker, Current_Balance(Ticker), "%\n Total Rate", Rate, "%", "%\n Rule 3")

                #SELL
                Upbit_Object.sell_market_order(Ticker, Current_Balance(Ticker))

        # SELL RULE # 4
        # RSI_HIGH
        elif ( 
            (DF['RSI'].iloc[-1] > 70) and
            (Current_Balance(Ticker) > 0)
            #(BP < Current_Balance(Ticker))
            ):

                ST = DF.index[-1]
                SP = Current_Balance(Ticker)

                Gain_ = SP - BP
                Rate_ = (((Current_Balance(Ticker) + (Current_Balance(Ticker) * 0.01 * Deal_Commission)) / BP) * 100)

                Gain += Gain_
                Rate += Rate_

                print("매도", ST, Ticker, Current_Balance(Ticker), "%\n Total Rate", Rate, "%", "%\n Rule 4")

                #SELL
                Upbit_Object.sell_market_order(Ticker, Current_Balance(Ticker))

        globals()['Gain_{}'. format(Ticker)] = Gain_
        globals()['Rate_{}'. format(Ticker)] = Rate_
        
        globals()['BT_{}'. format(Ticker)], globals()['BP_{}'. format(Ticker)] = BT, BP
        globals()['ST_{}'. format(Ticker)], globals()['SP_{}'. format(Ticker)] = ST, SP
            
        time.sleep(0.15)


In [ ]:
Current_Balance(Ticker)

0